# SITCOM-1120: Brake analysis

# Data
#### e-stop - rolex
 * 22-11-2023
 1.  Fisrt log TMA:  2023-11-22T19:56:33.622523   /// bquint@love01.cp.lsst.org  /
 2.  by 2023-11-23T00:51:28 / running the TMA Az/El breaks tests( E-stop ) /// ashugart@love01.cp.lsst.org 
 3.  by 2023-11-23T02:32:17.254532 /    bquint@love01.cp.lsst.org 
   * We ran another TMA Brake Test earlier today. Taking note now. 2023-11-23 00:38 UTC - 2023-11-23 00:42 UTC
   * TMA Brake tests in Azimuth - Going from 151º to 0º
   * We stopped around 102.5º, and we overshot until 89.67º
 4. 2023-11-23T03:08:50.50 /   bquint@love01.cp.lsst.org 
   * - 2023-11-23 01:02 UTC - 2023-11-23 01:05 UTC
   * - TMA Brake tests in Elevation - Going up from 20º to 80º.
   * - We stopped around 40º, and we overshooted until 45º.
 5.  2023-11-23 01:35 UTC - 2023-11-23 01:40 UTC / by bquint@love01.cp.lsst.org 2023-11-23T03:10:09
   * - TMA Brake tests in Elevation - Going down from 80º to 20º.
   * - We stopped around 57.8º, and we overshooted until 53.8º.
 * 




In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
import pandas as pd

from lsst_efd_client import EfdClient

client = EfdClient("usdf_efd")

%load_ext lab_black
%load_ext autoreload
%autoreload 2

async def process_data():
    try:
        data = await fetch_data()
        if data:
            print('Fetched data:', data)
    except Exception as e:
        print(f'Error processing data: {e}')

async def main():
    await process_data()

# Run the main function
asyncio.run(main())

In [ ]:
def addTime(t_start, t_end):
    start = Time(t_start, scale="utc")
    end = Time(t_end, scale="utc")
    return (start, end)


def accel(v, t):
    a = np.gradient(v, t)
    return a


def plotAz(az, acceleration=None):
    plt.figure(figsize=(16, 12))
    plt.suptitle("lsst.sal.MTMount.azimuth", fontsize=16)
    # Position
    plt.subplot(3, 1, 1)
    plt.plot(az["actualPosition"], label="Position")
    plt.plot(az["demandPosition"], label="demandPosition")
    plt.xlabel("Time")
    plt.ylabel("Posición")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # Velocity
    plt.subplot(3, 1, 2)
    plt.plot(az["actualVelocity"], label="Velocity")
    plt.plot(az["demandVelocity"], label="demandVelocity")
    plt.xlabel("Time")
    plt.ylabel("Velocity")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # Acceleration
    if acceleration is not None:
        plt.subplot(3, 1, 3)
        plt.plot(acceleration, label="Acceleration")
        plt.xlabel("Time")
        plt.ylabel("Acceleration")
        plt.legend()
        plt.grid(":", alpha=0.5)  # or alpha=0.3

    # disign
    plt.tight_layout()
    plt.show()
    return


def plotEl(el, acceleration=None):
    plt.figure(figsize=(16, 12))
    # Common title for all subplots
    plt.suptitle("lsst.sal.MTMount.elevation", fontsize=16)
    # Position
    plt.subplot(3, 1, 1)
    plt.plot(el["actualPosition"], label="Position")
    plt.plot(el["demandPosition"], label="demandPosition")
    plt.xlabel("Time")
    plt.ylabel("Posición")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # Velocity
    plt.subplot(3, 1, 2)
    plt.plot(el["actualVelocity"], label="Velocity")
    plt.plot(el["demandVelocity"], label="demandVelocity")
    plt.xlabel("Time")
    plt.ylabel("Velocity")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # Acceleration
    if acceleration is not None:
        plt.subplot(3, 1, 3)
        plt.plot(acceleration, label="Acceleration")
        plt.xlabel("Time")
        plt.ylabel("Acceleration")
        plt.legend()
        plt.ylim(10, -10)
        plt.grid(":", alpha=0.5)  # or alpha=0.3

    # disign
    # plt.grid(True, color="gray")
    plt.tight_layout()
    plt.show()
    return

# Running 

## Case 1:
 1.  Observing time started at : Fisrt log TMA:  2023-11-22T19:56:33.622523   /// bquint@love01.cp.lsst.org  /

In [ ]:
# obsTime = addTime("2023-11-22T18:00:00", "2023-11-23T06:00:00")
obsTime = addTime("2023-11-22T20:00:00", "2023-11-23T04:00:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

state = await client.select_time_series(
    "lsst.sal.MTMount.logevent_azimuthMotionState",
    [
        "state",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

logerror = await client.select_time_series(
    "lsst.sal.MTMount.logevent_error",
    [
        "active",
        "code",
        "latched",
        "subsystem",
        "text",
    ],
    obsTime[0],
    obsTime[1],
)

In [ ]:
df = pd.DataFrame(logerror)

In [ ]:
df

In [ ]:
# Convertir valores True y False a 1 y 0 respectivamente
df["active"] = df["active"].astype(int)

# Filtrar el DataFrame para obtener solo los valores 0 y 1
df_filter = df[df["active"].isin([0, 1])]

df_filter

In [ ]:
def errorLog(df_filter):
    plt.figure(figsize=(16, 12))
    # Common title for all subplots
    plt.suptitle("lsst.sal.MTMount.logevent_error", fontsize=16)
    # Error
    # plt.plot(df_filter["active"], label="active", )
    plt.eventplot(
        df_filter["active"],
        label="active",
    )
    # plt.plot(logerror["code"], label="code")
    # plt.plot(logerror["latched"], label="latched")
    # plt.plot(logerror["subsystem"], label="subsystem")
    # plt.plot(logerror["text"], label="text")
    plt.axvline(df_filter["active"])
    plt.xlabel("Time")
    plt.ylabel("Posición")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # disign
    plt.tight_layout()
    plt.show()
    return

In [ ]:
# errorLog(df_filter)

In [ ]:
# eventos = np.where(df_filter["active"] == 1)[0]
eventos = df_filter[df_filter["active"] == 1].index

plt.figure(figsize=(16, 12))
plt.suptitle("lsst.sal.MTMount.logevent_error", fontsize=16)

# Plot de datos de posición
plt.plot(az["actualPosition"], label="Position")
plt.plot(az["demandPosition"], label="demandPosition")

# Eventos
# Eventos
for evento in eventos:
    plt.axvline(x=evento, color="red", linestyle="--")

plt.xlabel("Time")
plt.ylabel("Posición")
plt.grid(":", alpha=0.5)
plt.legend()

# disign
plt.tight_layout()
plt.show()

In [ ]:
##I will pause the use of acceleration, after further discussion if it's needed.

# accAz = accel(az["actualVelocity"], az["timestamp"])
# accEl = accel(el["actualVelocity"], el["timestamp"])

state_0_1 = df[df['state'].isin([0, 1])]

# Plotea la serie temporal completa
plt.plot(state_0_1,  color='blue', label='Enum')

# Plotea los estados 0 y 1 con puntos rojos anchos
plt.scatter(estados_0_1.index, estados_0_1['datos'], color='red', s=100, label='Estados 0 y 1')

# Ajusta la apariencia del gráfico
plt.xlabel('Tiempo')
plt.ylabel('Valor')
plt.title('Gráfico con Estados 0 y 1 Marcados')
plt.legend()
plt.show()

In [ ]:
# plotAz(az, acceleration=None)
#plotAz(az, accAz)

In [ ]:
# plotEl(el, acceleration=None)
plotEl(el, accEl)

# Case 2:
 2.  by 2023-11-23T00:51:28 / running the TMA Az/El breaks tests( E-stop ) /// ashugart@love01.cp.lsst.org 


In [ ]:
obsTime = addTime("2023-11-23T00:30:00", "2023-11-23T01:05:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

accAz = accel(az["actualVelocity"], az["timestamp"])
accEl = accel(el["actualVelocity"], el["timestamp"])

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az, accAz)

In [ ]:
# plotEl(el, acceleration=None)
plotEl(el, accEl)

# Case 3:
 3.  by 2023-11-23T02:32:17.254532 /    bquint@love01.cp.lsst.org 
   * We ran another TMA Brake Test earlier today. Taking note now. 2023-11-23 00:38 UTC - 2023-11-23 00:42 UTC
   * TMA Brake tests in Azimuth - Going from 151º to 0º
   * We stopped around 102.5º, and we overshot until 89.67º


In [ ]:
obsTime = addTime("2023-11-23T00:30:00", "2023-11-23T00:50:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

accAz = accel(az["actualVelocity"], az["timestamp"])
accEl = accel(el["actualVelocity"], el["timestamp"])

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az, accAz)

In [ ]:
plotEl(el, accEl)

# Case 4:
 4. 2023-11-23T03:08:50.50 /   bquint@love01.cp.lsst.org 
   * - 2023-11-23 01:02 UTC - 2023-11-23 01:05 UTC
   * - TMA Brake tests in Elevation - Going up from 20º to 80º.
   * - We stopped around 40º, and we overshooted until 45º.


In [ ]:
obsTime = addTime("2023-11-23T00:55:00", "2023-11-23T01:10:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

accAz = accel(az["actualVelocity"], az["timestamp"])
accEl = accel(el["actualVelocity"], el["timestamp"])

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az, accAz)

In [ ]:
plotEl(el, accEl)

# Case 5:
 5.  2023-11-23 01:35 UTC - 2023-11-23 01:40 UTC / by bquint@love01.cp.lsst.org 2023-11-23T03:10:09
   * - TMA Brake tests in Elevation - Going down from 80º to 20º.
   * - We stopped around 57.8º, and we overshooted until 53.8º.

In [ ]:
obsTime = addTime("2023-11-23T00:30:00", "2023-11-23T01:00:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

accAz = accel(az["actualVelocity"], az["timestamp"])
accEl = accel(el["actualVelocity"], el["timestamp"])

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az, accAz)

In [ ]:
plotEl(el, accEl)

## Next:
1. Summary state error , search for logs .. each one
2. Calculate distance after e-stop
3. Search in GIS. / Search for FAULT 
4. Search for cmd_fault an E-Stop words?
5. Determind by diff ways the distance, -
6. Do it automatics the searsh of e-stops